# Introduction/Business Problem 
# Analyze Steakhouse vs Vegetarian / Vegan Restaurant Concentration in New York (Brooklyn, Manhattan)
- Business problem: Decide which New York neighborhoods to target for a PETA (People for the Ethical Treatment of Animals) animal awareness campaign
- As a proxy, we will compare the # of Steakhouses vs  Vegetarian / Vegan Restaurant in various NY neighborhoods, trying to see if there are obvious geographical patterns for fellow animal supporters

# Data Section
- For NYC geographical data, we will first use the NYU library's "2014 New York City Neighborhood Names". It contains the name each NYC neighborhood
- Next, we will find the coordintes of the neighborhoods via geopy.geocoders
- We will the load NYC neighborhood data into a Pandas dataframe
- For contextual data (steakhouse vs veg), we will use the Foursquare API, filtering via category codes


- For this study, we will focus on the boroughs of Manhattan and Brooklyn

In [24]:
# starting point code = 
# Coursea // IBM Data Science Professional Certificate // Course 9 // Week 3) // " Neighborhoods in New York City"

import pandas as pd
import urllib.request, json

borough_analyze = ("Manhattan", "Brooklyn")

with urllib.request.urlopen("https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json") as url:
    newyork_data = json.load(url)
    neighborhoods_data = newyork_data['features']
    
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    
    if borough in borough_analyze:
        neighborhood_name = data['properties']['name']

        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]

        neighborhoods = neighborhoods.append({'Borough': borough,
                                              'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)
         
neighborhood_count_manhattan = sum(neighborhoods['Borough'] == "Manhattan")
print(f"Manhattan has {neighborhood_count_manhattan} neighborhoods")

neighborhood_count_brooklyn = sum(neighborhoods['Borough'] == "Brooklyn")
print(f"Brooklyn has {neighborhood_count_brooklyn} neighborhoods")

Manhattan has 40 neighborhoods
Brooklyn has 70 neighborhoods


# Visualization setup

- Get coordinates of Manhattan / Brooklyn
- Populate raw Folium map

In [37]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address_Manhattan = 'Manhattan, NY'
address_Brooklyn = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_Manattan = geolocator.geocode(address_Manhattan)
location_Brooklyn = geolocator.geocode(address_Brooklyn)

latitude = (location_Manattan.latitude + location_Brooklyn.latitude)/2
longitude = (location_Manattan.longitude + location_Brooklyn.longitude)/2
print('The geograpical coordinate of Manhattan/Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan/Brooklyn are 40.720095349999994, -73.9547059.


In [136]:
import folium

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Request and compute Foursquare data

- Filter in only  the neighborhoods with the most boutiques (at least 10)

- Then display the top 3 most popular Boutique (cloth shopping) options


In [216]:
import json, requests

# CBJ1TCAYGC2K43HEI5CBX5WF5IYQU41YCQFXYUJSDRV34JTW
# XQOH02ZW2L2KW04WAEXXU2U5FLCM4QDH5NSQIHKA1QZBFQEX

# B0LPCPNQLDCNHSV3KLKMLY0XMKD0NGH40ZJZM2DSM3FNHBTC
# 2SJFKJTXBG4D21XFM5S2JP5IZRMV0LVOQ3UZRP0YLDWVHJW3
def getNearbyVenues(neighborhoods, category_id, radius=1000, limit=200):
    venues_list=[]
    for row in neighborhoods.itertuples():
        base_url = 'https://api.foursquare.com/v2/venues/explore?&'
        url = '{}client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            base_url,
            '0IYK3MFUCQLTP4W4Q3E1ESKOQJ4SVVADK1XTAUCBLPUXXSTG', 
            '0XGXDKOSWKS1HLRB5PIFTGYRFJLFRN2OLD4NV2FSSCCOM21I', 
            '20180323', 
            row.Latitude, 
            row.Longitude, 
            radius,
            limit,
            category_id)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        if(not results):
            print(row.Borough, "//", row.Neighborhood, "has 0 result")
        else:
            print(row.Borough, "//", row.Neighborhood, "has", len(results), "result")
            for v in results:
                # return only relevant information for each nearby venue
                try:
                    category = v['venue']['categories'][0]['name']
                    if (category=="Vegetarian / Vegan Restaurant" or category=="Steakhouse"):
                        venues_list.append((
                            row.Borough,
                            row.Neighborhood, 
                            row.Latitude, 
                            row.Longitude, 
                            v['venue']['name'], 
                            v['venue']['location']['lat'], 
                            v['venue']['location']['lng'],  
                            v['venue']['categories'][0]['name']))
                except:
                    continue

    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = [
            'Borough',
            'Neighborhood', 
            'Neighborhood Latitude', 
            'Neighborhood Longitude', 
            'Venue', 
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category']
    
    
    return(nearby_venues)

category_steak_house = "4bf58dd8d48988d1cc941735"
category_veg = "4bf58dd8d48988d1d3941735"
venues = getNearbyVenues(neighborhoods, f"{category_steak_house},{category_veg}")

Manhattan // Marble Hill has 2 result
Brooklyn // Bay Ridge has 3 result
Brooklyn // Bensonhurst has 0 result
Brooklyn // Sunset Park has 3 result
Brooklyn // Greenpoint has 9 result
Brooklyn // Gravesend has 0 result
Brooklyn // Brighton Beach has 0 result
Brooklyn // Sheepshead Bay has 3 result
Brooklyn // Manhattan Terrace has 6 result
Brooklyn // Flatbush has 4 result
Brooklyn // Crown Heights has 4 result
Brooklyn // East Flatbush has 0 result
Brooklyn // Kensington has 3 result
Brooklyn // Windsor Terrace has 3 result
Brooklyn // Prospect Heights has 10 result
Brooklyn // Brownsville has 0 result
Brooklyn // Williamsburg has 13 result
Brooklyn // Bushwick has 7 result
Brooklyn // Bedford Stuyvesant has 4 result
Brooklyn // Brooklyn Heights has 8 result
Brooklyn // Cobble Hill has 4 result
Brooklyn // Carroll Gardens has 4 result
Brooklyn // Red Hook has 0 result
Brooklyn // Gowanus has 3 result
Brooklyn // Fort Greene has 8 result
Brooklyn // Park Slope has 8 result
Brooklyn // C

# Methodology - Data Analysis
- We need to set a threshold for minimum number of results, as some neighborhoods doesn't have many steakhouses or veg restaurants. Looking at the sorted # of results, it seems 10 is a reasonable cutoff.
- If a neighborhood has less than 5 Vegetarian / Vegan Restaurants and less than 5 Steakhouses, we consider the neighborhood "undecided" and drop it from the analysis. This faciliates one-hot processing / normalization, as it gives equal-weight to each neighboorhood (e.g., 0 veg + 1 steak has same one-hot output as 0 veg + 100 steak).

In [321]:
display(venues.head())

Borough Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Manhattan  Marble Hill              40.876551              -73.910660   
1  Manhattan  Marble Hill              40.876551              -73.910660   
2   Brooklyn    Bay Ridge              40.625801              -74.030621   
3   Brooklyn    Bay Ridge              40.625801              -74.030621   
4   Brooklyn  Sunset Park              40.645103              -74.010316   

                                   Venue  Venue Latitude  Venue Longitude  \
0                        Parrilla Latina       40.877473       -73.906073   
1  Kingsbridge-Riverdale Farmers' Market       40.879394       -73.907125   
2                  Shangri-La Vegetarian       40.632136       -74.027573   
3                        Three's A Crowd       40.618227       -74.030480   
4                       Lucky Vegetarian       40.640570       -74.004262   

                  Venue Category  
0                     Steakhouse  
1  Vegetarian / Vegan Restaurant  
2  Vegetarian / Vegan Restaurant  
3  Vegetarian / Vegan Restaurant  
4  Vegetarian / Vegan Restaurant

In [359]:
import numpy as np

min_count = 5
venues_grouped = pd.pivot_table(venues[["Neighborhood", "Venue Category"]], index="Neighborhood", columns="Venue Category",
                                aggfunc= np.size, fill_value=0)

venues_grouped_filtered = venues_grouped[(venues_grouped["Steakhouse"] >= min_count) | (venues_grouped["Vegetarian / Vegan Restaurant"] >= min_count)]

print("Before applying min_count =", min_count, " there are", len(venues_grouped.index), "neighborhoods")
print("Before applying min_count =", min_count, " there are", len(venues.index), "venues")
display(venues_grouped.head().reset_index())

neighborhoods_kept = venues_grouped_filtered.index
venues_filtered = venues[venues["Neighborhood"].isin(neighborhoods_kept)]

print("After applying min_count =", min_count, " there are", len(venues_grouped_filtered.index), "neighborhoods")
print("After applying min_count =", min_count, " there are", len(venues_filtered.index), "venues")

onehot = pd.get_dummies(venues_filtered[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues_filtered['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

venues_filtered_grouped = onehot.groupby('Neighborhood').mean().reset_index()

display(venues_filtered_grouped.head())

Before applying min_count = 5  there are 89 neighborhoods
Before applying min_count = 5  there are 1257 venues


Venue Category        Neighborhood  Steakhouse  Vegetarian / Vegan Restaurant
0                       Bath Beach           2                              2
1                Battery Park City          24                             12
2                        Bay Ridge           0                              4
3               Bedford Stuyvesant           0                              8
4                      Boerum Hill           2                             18

After applying min_count = 5  there are 61 neighborhoods
After applying min_count = 5  there are 1200 venues


Neighborhood  Steakhouse  Vegetarian / Vegan Restaurant
0   Battery Park City    0.666667                       0.333333
1  Bedford Stuyvesant    0.000000                       1.000000
2         Boerum Hill    0.100000                       0.900000
3    Brooklyn Heights    0.375000                       0.625000
4            Bushwick    0.000000                       1.000000

# Methodology: Model Selection
- We apply a KMeans(n=3) Clustering analysis
- We tried various valuse (n=3..7) and noticed that there are really just three big groups, namely...
- Group 1 = "More steakhouse than veg";
- Group 2 = "Mostly veg";
- Group 3...n = "More veg than steakhouses"


- With the above observation, it seems n=3 is a good choice

In [360]:
from sklearn.cluster import KMeans

kclusters = 3

venues_filtered_grouped_clustering = venues_filtered_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_filtered_grouped_clustering)   # run k-means clustering
venues_filtered_grouped.insert(0, 'Cluster Labels', kmeans.labels_)   # add clustering labels

venues_filtered_merged = venues_filtered

# merge cluster label with list of venues 
venues_filtered_merged = venues_filtered_merged.join(venues_filtered_grouped.set_index('Neighborhood'), on='Neighborhood')
venues_filtered_merged.sort_values('Cluster Labels', inplace=True)
venues_filtered_merged.reset_index(inplace=True, drop=True)

# translate meaning of cluster labels
venues_filtered_grouped = venues_filtered_grouped.sort_values("Cluster Labels").reset_index(drop=True)
cluster_groups = venues_filtered_grouped.groupby('Cluster Labels').mean().reset_index()
display(cluster_groups)
display(venues_filtered_grouped)

Cluster Labels  Steakhouse  Vegetarian / Vegan Restaurant
0               0    0.680068                       0.319932
1               1    0.068858                       0.931142
2               2    0.309218                       0.690782

Cluster Labels               Neighborhood  Steakhouse  \
0                0          Battery Park City    0.666667   
1                0                 Turtle Bay    0.700000   
2                0                 Tudor City    0.642857   
3                0               Sutton Place    0.625000   
4                0              Ocean Parkway    0.777778   
5                0                Murray Hill    0.562500   
6                0                    Midtown    0.622222   
7                0          Manhattan Terrace    0.833333   
8                0               Hudson Yards    0.652174   
9                0                  Homecrest    1.000000   
10               0         Financial District    0.545455   
11               0             Lincoln Square    0.600000   
12               0                    Clinton    0.612903   
13               1              Crown Heights    0.000000   
14               1                       Noho    0.140000   
15               1                 North Side    0.142857   
16               1                   Bushwick    0.000000   
17               1                 Park Slope    0.000000   
18               1           Prospect Heights    0.000000   
19               1  Prospect Lefferts Gardens    0.000000   
20               1        Prospect Park South    0.000000   
21               1                      Rugby    0.000000   
22               1                 South Side    0.150000   
23               1            Stuyvesant Town    0.111111   
24               1                Sunset Park    0.000000   
25               1                Boerum Hill    0.100000   
26               1         Bedford Stuyvesant    0.000000   
27               1               Williamsburg    0.166667   
28               1            Lower East Side    0.142857   
29               1            Windsor Terrace    0.000000   
..             ...                        ...         ...   
31               1                  Yorkville    0.142857   
32               1                  Chinatown    0.181818   
33               1           Hamilton Heights    0.000000   
34               1               East Village    0.138889   
35               1          East Williamsburg    0.000000   
36               2                   Downtown    0.272727   
37               2               West Village    0.282051   
38               2               Vinegar Hill    0.333333   
39               2            Upper West Side    0.250000   
40               2            Upper East Side    0.461538   
41               2                      Dumbo    0.333333   
42               2                East Harlem    0.333333   
43               2                    Tribeca    0.454545   
44               2           Brooklyn Heights    0.375000   
45               2                    Erasmus    0.250000   
46               2               Little Italy    0.216216   
47               2                 Lenox Hill    0.409091   
48               2           Roosevelt Island    0.200000   
49               2                Fort Greene    0.250000   
50               2               Fulton Ferry    0.285714   
51               2                   Gramercy    0.277778   
52               2                 Greenpoint    0.250000   
53               2          Greenwich Village    0.228070   
54               2               Civic Center    0.482759   
55               2                Cobble Hill    0.250000   
56               2              Midtown South    0.471429   
57               2             Central Harlem    0.200000   
58               2                    Chelsea    0.310345   
59               2                       Soho    0.260870   
60               2                   Flatiron    0.292308   

    Vegetarian / Vegan Restaurant  
0                        0.333333  
1                        0.300000  
2                        0.357143  
3                        0.375000  
4                        0.222222  
5    

# Results: Visualization & Conclusion

- We found that there are many areas with predominately veg places with little to no steakhouses (group 1). This makes up 23 of the 61 nighborhoods.
- We didn't find the reverse - namely, areas with mostly steakhouses and little veg places except for two areas (Manattan Terrace, Homecrest)

- As noted in the previous section, the map shows a relatively contiguous clustering...
- Group 1 (red) = "More steakhouse than veg";
- Group 2 (blue) = "Mostly veg";
- Group 3 (green) = "More veg than steakhouses"


- The clustering shows a very clear pattern
- Group 1 concentrates in just two areas: midtown Manhattan, the Financial District of Manhattan and southern Brooklyn. The two Manhattan neighborhoods happens to be where financial companies are heavily concentrated, and are thur frequent "corproate dinner" destinations.
- The rest of Manhattan is composed of Group 3. Given that generally there are in general more veg places than steakhouses, this is likely a neutral composition and simply reflects a balance of preferences. Also keep in mind that steakhouses are generally far larger establishments than veg spots, so the # of patrons served is closer to 40/60 than 30/70 as indicated in the previous section.
- Downtown Brooklyn (immediate southern area over the Manhattan bridge) also comprises of group 3. This likely reflects the relatively higher income of the area, as that region serves as homes for many Manhattan commuters. 
- Group 2 is composed of the rest of Brooklyn, including young and trendy neighborhoods like Williamsburg, Greenpoint, and Bushwick. More surprising is that this trend extends to the rest of Brooklyn. This likely reflects not so much a high concentration of veg places than a lack of steakhouses.


- With the data in mind, our suggestion for the PETA animal awareness campaign is to target business addresses in Manhattan Midtown + Finacial District, as well as residential areas of downtown Brooklyn, and possibly the (Russian) ethnic neighborhood in southern Brooklyn 

In [362]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_filtered_merged['Venue Latitude'], venues_filtered_merged['Venue Longitude'], venues_filtered_merged['Neighborhood'], venues_filtered_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters